# PyCitySchools Analysis
-  Observable Trend 1 - Based on Overall Passing Rate, the top performing schools were small to medium sized charter schools.
-  Observable Trend 2 - Based on Overall Passing Rate, the bottom performing schools were large, district schools.
-  Observable Trend 3 - Math and Reading Scores by Grade showed no significant variance or trends.



In [24]:
import pandas as pd
import numpy as np
import os

In [25]:
school_data = os.path.join('schools_complete.csv')
student_data = os.path.join('students_complete.csv')

school_data_pd = pd.read_csv(school_data)
student_data_pd = pd.read_csv(student_data)
school_data_pd = school_data_pd.rename(columns={'name':'school_name'})
student_data_pd = student_data_pd.rename(columns={'school':'school_name'})
district_data_pd = pd.merge(student_data_pd, school_data_pd, how="inner")
#district_data_pd.head(5)

## District Summary

In [26]:
Columns = ['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 
           'Average Reading Score', '% Passing Math', '% Passing Reading', "% Overall Passing Rate"]

total_schools = district_data_pd['school_name'].nunique()
total_students = district_data_pd['Student ID'].nunique()
total_budget = school_data_pd['budget'].sum()
avg_math_score = district_data_pd['math_score'].mean()
avg_reading_score = district_data_pd['reading_score'].mean()

passing_math_count = district_data_pd[(district_data_pd['math_score']>70)].count()['name']
passing_reading_count = district_data_pd[(district_data_pd['reading_score']>70)].count()['name']

           
perc_passing_math = (passing_math_count/total_students) * 100
perc_passing_reading = (passing_reading_count/total_students) * 100
perc_passing_overall = (perc_passing_math+perc_passing_reading)/2

Values = [total_schools, total_students, total_budget, avg_math_score, avg_reading_score, 
          perc_passing_math, perc_passing_reading, perc_passing_overall]

District_Summary = pd.DataFrame([Values], columns=Columns)
District_Summary = District_Summary.round(2)
District_Summary['Total Budget'] = District_Summary['Total Budget'].map("${:,.0f}".format)

District_Summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.99,81.88,72.39,82.97,77.68


## School Summary

In [27]:
stud_grpby_school_pd = district_data_pd.groupby(['school_name'], as_index=False)
school_avg_math_pd = pd.DataFrame(stud_grpby_school_pd['math_score'].mean())
school_avg_reading_pd = pd.DataFrame(stud_grpby_school_pd['reading_score'].mean())
#avg_school_math_pd = stud_grpby_school_pd[stud_grpby_school_pd['math_score'].mean()].groupby(['school_name'], as_index=False)
school_avg_math_pd.columns = ['school_name', 'avg_school_math']
#avg_school_reading_pd = stud_grpby_school_pd[stud_grpby_school_pd['reading_score'].mean()].groupby(['school_name'], as_index=False)
school_avg_reading_pd.columns = ['school_name', 'avg_school_reading']                         

grp_math_pass_pd = student_data_pd[student_data_pd['math_score']>=70].groupby(['school_name'], as_index=False)
school_math_pass_pd = pd.DataFrame(grp_math_pass_pd['math_score'].count())
school_math_pass_pd.columns = ['school_name', 'math_pass_count']
#school_math_pass_pd.head()
grp_reading_pass_pd = student_data_pd[student_data_pd['reading_score']>=70].groupby(['school_name'], as_index=False)
school_reading_pass_pd = pd.DataFrame(grp_reading_pass_pd['reading_score'].count())
school_reading_pass_pd.columns = ['school_name', 'reading_pass_count']
#school_reading_pass_pd.head()


merged_school_data_pd = pd.merge(school_data_pd, school_avg_math_pd, on='school_name')
merged_school_data_pd = pd.merge(merged_school_data_pd, school_math_pass_pd, on='school_name')
merged_school_data_pd = pd.merge(merged_school_data_pd, school_avg_reading_pd, on='school_name')
merged_school_data_pd = pd.merge(merged_school_data_pd, school_reading_pass_pd, on='school_name')



merged_school_data_pd["perc_math_pass"] = (merged_school_data_pd['math_pass_count']/merged_school_data_pd['size'])*100
merged_school_data_pd["perc_reading_pass"] = (merged_school_data_pd['reading_pass_count']/merged_school_data_pd['size'])*100
merged_school_data_pd["perc_pass_overall"] = (merged_school_data_pd['perc_math_pass']+merged_school_data_pd['perc_reading_pass'])/2
merged_school_data_pd["per_student_budget"] = (merged_school_data_pd['budget']/merged_school_data_pd['size'])


school_summary_data = pd.DataFrame(merged_school_data_pd[['school_name', 'type', 'size', 'budget', 'per_student_budget', 
                                                     'avg_school_math', 'avg_school_reading', 'perc_math_pass', 
                                                     'perc_reading_pass', 'perc_pass_overall']])
school_summary_data.columns = ['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 
                               'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                               '% Overall Passing Rate']

school_summary = pd.DataFrame(merged_school_data_pd[['school_name', 'type', 'size', 'budget', 'per_student_budget', 
                                                     'avg_school_math', 'avg_school_reading', 'perc_math_pass', 
                                                     'perc_reading_pass', 'perc_pass_overall']])

school_summary.columns = ['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 
                               'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                               '% Overall Passing Rate']

#school_summary_data.head(15)
school_summary['Total School Budget'] = school_summary_data['Total School Budget'].map("${:,.2f}".format)
school_summary['Per Student Budget'] = school_summary_data['Per Student Budget'].map("${:,.2f}".format)
school_summary


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

In [28]:
sort_school_summary_data = school_summary_data.sort_values('% Overall Passing Rate', ascending=False)[:5]
sort_school_summary_data.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [29]:
sort_school_summary_data = school_summary_data.sort_values('% Overall Passing Rate')[:5]
sort_school_summary_data

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [30]:
ninth_pd = student_data_pd.loc[student_data_pd['grade'] == "9th"].groupby('school_name', as_index=False)
tenth_pd = student_data_pd.loc[student_data_pd['grade'] == "10th"].groupby('school_name', as_index=False)
eleventh_pd = student_data_pd.loc[student_data_pd['grade'] == "11th"].groupby('school_name', as_index=False)
twelfth_pd = student_data_pd.loc[student_data_pd['grade'] == "12th"].groupby('school_name', as_index=False)

ninth_avg_math = pd.DataFrame(ninth_pd['math_score'].mean())
tenth_avg_math = pd.DataFrame(tenth_pd['math_score'].mean())
eleventh_avg_math = pd.DataFrame(eleventh_pd['math_score'].mean())
twelfth_avg_math = pd.DataFrame(twelfth_pd['math_score'].mean())

math_grade_pd = pd.merge(ninth_avg_math, tenth_avg_math, on='school_name')
math_grade_pd = pd.merge(math_grade_pd, eleventh_avg_math, on='school_name')
math_grade_pd = pd.merge(math_grade_pd, twelfth_avg_math, on='school_name')
math_grade_pd.columns = ['School Name', '9th', '10th', '11th', '12th']
math_grade_pd.head(15)

,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [31]:
ninth_avg_read = pd.DataFrame(ninth_pd['reading_score'].mean())
tenth_avg_read = pd.DataFrame(tenth_pd['reading_score'].mean())
eleventh_avg_read = pd.DataFrame(eleventh_pd['reading_score'].mean())
twelfth_avg_read = pd.DataFrame(twelfth_pd['reading_score'].mean())

read_grade_pd = pd.merge(ninth_avg_read, tenth_avg_read, on='school_name')
read_grade_pd = pd.merge(read_grade_pd, eleventh_avg_read, on='school_name')
read_grade_pd = pd.merge(read_grade_pd, twelfth_avg_read, on='school_name')
read_grade_pd.columns = ['School Name', '9th', '10th', '11th', '12th']
read_grade_pd.head(15)

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [32]:
bins = [0, 585, 615, 645, 675]
bin_names = ["$0-585", "$585-615", "$615-645", "$645-675"]
score_by_budget = school_summary_data[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                      '% Overall Passing Rate']].groupby(pd.cut(school_summary_data['Per Student Budget'], 
                                                                               bins=bins, labels=bin_names)).mean()
score_by_budget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
$0-585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [33]:
bins = [0, 1000, 2000, 5000]
bin_names = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
score_by_size = school_summary_data[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                      '% Overall Passing Rate']].groupby(pd.cut(school_summary_data['Total Students'], 
                                                                               bins=bins, labels=bin_names)).mean()
score_by_size.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [34]:
school_summary_type = school_summary_data
school_summary_type["School Type"] = school_summary_type["School Type"].replace({"Charter":1, "District": 2})
bins = [0, 1, 2]
bin_names = ["Charter", "District"]
score_by_type = school_summary_type[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                      '% Overall Passing Rate']].groupby(pd.cut(school_summary_type['School Type'], 
                                                                               bins=bins, labels=bin_names)).mean()
score_by_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
